In [2]:

import pandas as pd
import csv
import os
from sqlalchemy import create_engine 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from pymongo import MongoClient


In [56]:
path = os.path.join("data/visitors_2020.csv")
visitorCount_20 = pd.read_csv(path, header=[7], encoding='utf-8', delimiter=",", index_col=[0,1], keep_default_na=False, na_filter=True)
visitorCount_20 = visitorCount_20.dropna(axis=1, how='all', thresh=3, inplace=False)

visitorCount_20.head(100)



Recreation Visitors Percent Change  \
State   Park                                                            
Alabama                                                                 
        Horseshoe Bend NMP                      28,600        -36.97%   
        Little River Canyon NPRES              802,375         23.44%   
        Natchez Trace PKWY                     428,736         -2.72%   
        Russell Cave NM                          8,462        -49.15%   
...                                                ...            ...   
        Great Sand Dunes NP & PRES             461,532        -12.51%   
        Hovenweep NM                             8,736        -43.92%   
        Mesa Verde NP                          287,477        -48.31%   
        Rocky Mountain NP                    3,305,199        -29.23%   
        Sand Creek Massacre NHS                  4,215        -26.07%   

                                   Recreation Visitor Days Percent Change.1  \
State   Park                                                                  
Alabama                                                                       
        Horseshoe Bend NMP                           8,788          -25.20%   
        Little River Canyon NPRES                  168,827           27.60%   
        Natchez Trace PKWY                         143,413           -3.01%   
        Russell Cave NM                              1,074          -55.36%   
...                                                    ...              ...   
        Great Sand Dunes NP & PRES                 159,822          -29.39%   
        Hovenweep NM                                 2,350          -48.12%   
        Mesa Verde NP                              156,122          -52.80%   
        Rocky Mountain NP                        1,799,117          -31.54%   
        Sand Creek Massacre NHS                        702          -26.11%   

                                   Unnamed: 6 Unnamed: 7  \
State   Park                                               
Alabama                                                    
        Horseshoe Bend NMP                                 
        Little River Canyon NPRES                          
        Natchez Trace PKWY                                 
        Russell Cave NM                                    
...                                       ...        ...   
        Great Sand Dunes NP & PRES                         
        Hovenweep NM                                       
        Mesa Verde NP                                      
        Rocky Mountain NP                                  
        Sand Creek Massacre NHS                            

                                   Non-recreation Visitors Percent Change.2  \
State   Park                                                                  
Alabama                                                                       
        Horseshoe Bend NMP                               0                    
        Little River Canyon NPRES                        0                    
        Natchez Trace PKWY                         583,028           -9.08%   
        Russell Cave NM                                  0                    
...                                                    ...              ...   
        Great Sand Dunes NP & PRES                     470            0.00%   
        Hovenweep NM                                   132            0.00%   
        Mesa Verde NP                                7,519          -25.41%   
        Rocky Mountain NP                            8,750            0.00%   
        Sand Creek Massacre NHS                          0                    

                                   Recreation Visitor Hours Percent Change.3  \
State   Park                                                                   
Alabama                                                                        
        

In [51]:
visitorCount_20.columns

Index(['Recreation Visitors', 'Percent Change', 'Recreation Visitor Days',
       'Percent Change.1', 'Unnamed: 6', 'Unnamed: 7',
       'Non-recreation Visitors', 'Percent Change.2',
       'Recreation Visitor Hours', 'Percent Change.3',
       'Non-recreation Visitor Hours', 'Percent Change.4',
       'Concessioner\nLodging Overnights', 'Percent Change.5',
       'Concessioner\nCamping Overnights', 'Percent Change.6',
       'Tent Overnights', 'Percent Change.7', 'Recreation Vehicle Overnights',
       'Percent Change.8', 'Back Country\nOvernights', 'Percent Change.9',
       'Non-recreation Overnights', 'Percent Change.10', 'Misc. Overnights',
       'Percent Change.11'],
      dtype='object')

In [5]:
# tracounts = tracounts.set_index('UnitCode')
# tracounts = tracounts[['ParkName', 'ParkType', 'Region', 'Year', 'Month','TrafficCount', 'TrafficCountTotal.1']].copy()
# tracounts = tracounts.rename(columns={"TrafficCountTotal.1": "TrafficCountTotal"})
# tracounts.head(30)

In [52]:
visitorCount_20_select = visitorCount_20[['Recreation Visitors', 'Percent Change', 'Recreation Visitor Days',
       'Percent Change.1', 'Non-recreation Visitors', 'Percent Change.2',
       'Recreation Visitor Hours', 'Percent Change.3',
       'Non-recreation Visitor Hours', 'Percent Change.4',]]

visitorCount_20_select

Recreation Visitors Percent Change  \
State   Park                                                                   
Alabama                                                                        
        Horseshoe Bend NMP                             28,600        -36.97%   
        Little River Canyon NPRES                     802,375         23.44%   
        Natchez Trace PKWY                            428,736         -2.72%   
        Russell Cave NM                                 8,462        -49.15%   
...                                                       ...            ...   
        Grand Teton NP                              3,289,638         -3.41%   
        John D. Rockefeller, Jr. MEM PKWY           1,373,817         -3.70%   
        Yellowstone NP                              1,865,088         -5.32%   
        Subtotal Wyoming                            7,089,437         -4.60%   
Total   Grand Total                               237,064,318        -27.62%   

                                          Recreation Visitor Days  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                  8,788   
        Little River Canyon NPRES                         168,827   
        Natchez Trace PKWY                                143,413   
        Russell Cave NM                                     1,074   
...                                                           ...   
        Grand Teton NP                                  1,486,410   
        John D. Rockefeller, Jr. MEM PKWY                 144,764   
        Yellowstone NP                                  3,125,466   
        Subtotal Wyoming                                4,883,337   
Total   Grand Total                                    87,912,527   

                                          Percent Change.1  \
State   Park                                                 
Alabama                                                      
        Horseshoe Bend NMP                         -25.20%   
        Little River Canyon NPRES                   27.60%   
        Natchez Trace PKWY                          -3.01%   
        Russell Cave NM                            -55.36%   
...                                                    ...   
        Grand Teton NP                             -15.87%   
        John D. Rockefeller, Jr. MEM PKWY            2.19%   
        Yellowstone NP                             -32.55%   
        Subtotal Wyoming                           -27.08%   
Total   Grand Total                                -26.23%   

                                          Non-recreation Visitors  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                      0   
        Little River Canyon NPRES                               0   
        Natchez Trace PKWY                                583,028   
        Russell Cave NM                                         0   
...                                                           ...   
        Grand Teton NP                                  1,220,029   
        John D. Rockefeller, Jr. MEM PKWY                  10,800   
        Yellowstone NP                                  1,099,227   
        Subtotal Wyoming                                2,334,598   
Total   Grand Total                                   148,436,337   

                                          Percent Change.2  \
State   Park                                                 
Alabama                                                      
        Horseshoe Bend NMP                                   
        Little River Canyon NPRES                            
        Natchez Trace PKWY                          -9.08%   
        Russell Cave NM   

In [98]:
visCount_20 = visitorCount_20_select[['Recreation Visitors', 'Recreation Visitor Days',
       'Non-recreation Visitors',
       'Recreation Visitor Hours',
       'Non-recreation Visitor Hours']]
visCount_20

Recreation Visitors  \
State   Park                                                    
Alabama                                                         
        Horseshoe Bend NMP                             28,600   
        Little River Canyon NPRES                     802,375   
        Natchez Trace PKWY                            428,736   
        Russell Cave NM                                 8,462   
...                                                       ...   
        Grand Teton NP                              3,289,638   
        John D. Rockefeller, Jr. MEM PKWY           1,373,817   
        Yellowstone NP                              1,865,088   
        Subtotal Wyoming                            7,089,437   
Total   Grand Total                               237,064,318   

                                          Recreation Visitor Days  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                  8,788   
        Little River Canyon NPRES                         168,827   
        Natchez Trace PKWY                                143,413   
        Russell Cave NM                                     1,074   
...                                                           ...   
        Grand Teton NP                                  1,486,410   
        John D. Rockefeller, Jr. MEM PKWY                 144,764   
        Yellowstone NP                                  3,125,466   
        Subtotal Wyoming                                4,883,337   
Total   Grand Total                                    87,912,527   

                                          Non-recreation Visitors  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                      0   
        Little River Canyon NPRES                               0   
        Natchez Trace PKWY                                583,028   
        Russell Cave NM                                         0   
...                                                           ...   
        Grand Teton NP                                  1,220,029   
        John D. Rockefeller, Jr. MEM PKWY                  10,800   
        Yellowstone NP                                  1,099,227   
        Subtotal Wyoming                                2,334,598   
Total   Grand Total                                   148,436,337   

                                          Recreation Visitor Hours  \
State   Park                                                         
Alabama                                                              
        Horseshoe Bend NMP                                 105,461   
        Little River Canyon NPRES                        2,025,934   
        Natchez Trace PKWY                               1,720,966   
        Russell Cave NM                                     12,894   
...                                                            ...   
        Grand Teton NP                                  17,836,930   
        John D. Rockefeller, Jr. MEM PKWY                1,737,179   
        Yellowstone NP                                  37,505,599   
        Subtotal Wyoming                                58,600,107   
Total   Grand Total                                  1,054,952,561   

                                          Non-recreation Visitor Hours  
State   Park                                                            
Alabama                                                                 
        Horseshoe Bend NMP                                           0  
        Little River Canyon NPRES                                    0  
        Natchez Trace PKWY                                     583,028  
        Russell Cave NM                            

In [57]:
path = os.path.join("data/visitors_2019.csv")
visitorCount_19 = pd.read_csv(path, header=[7], encoding='utf-8', delimiter=",", index_col=[0,1], keep_default_na=False, na_filter=True)
visitorCount_19 = visitorCount_20.dropna(axis=1, how='all', thresh=3, inplace=False)

visitorCount_19.head(100)


Recreation Visitors Percent Change  \
State   Park                                                            
Alabama                                                                 
        Horseshoe Bend NMP                      28,600        -36.97%   
        Little River Canyon NPRES              802,375         23.44%   
        Natchez Trace PKWY                     428,736         -2.72%   
        Russell Cave NM                          8,462        -49.15%   
...                                                ...            ...   
        Great Sand Dunes NP & PRES             461,532        -12.51%   
        Hovenweep NM                             8,736        -43.92%   
        Mesa Verde NP                          287,477        -48.31%   
        Rocky Mountain NP                    3,305,199        -29.23%   
        Sand Creek Massacre NHS                  4,215        -26.07%   

                                   Recreation Visitor Days Percent Change.1  \
State   Park                                                                  
Alabama                                                                       
        Horseshoe Bend NMP                           8,788          -25.20%   
        Little River Canyon NPRES                  168,827           27.60%   
        Natchez Trace PKWY                         143,413           -3.01%   
        Russell Cave NM                              1,074          -55.36%   
...                                                    ...              ...   
        Great Sand Dunes NP & PRES                 159,822          -29.39%   
        Hovenweep NM                                 2,350          -48.12%   
        Mesa Verde NP                              156,122          -52.80%   
        Rocky Mountain NP                        1,799,117          -31.54%   
        Sand Creek Massacre NHS                        702          -26.11%   

                                   Unnamed: 6 Unnamed: 7  \
State   Park                                               
Alabama                                                    
        Horseshoe Bend NMP                                 
        Little River Canyon NPRES                          
        Natchez Trace PKWY                                 
        Russell Cave NM                                    
...                                       ...        ...   
        Great Sand Dunes NP & PRES                         
        Hovenweep NM                                       
        Mesa Verde NP                                      
        Rocky Mountain NP                                  
        Sand Creek Massacre NHS                            

                                   Non-recreation Visitors Percent Change.2  \
State   Park                                                                  
Alabama                                                                       
        Horseshoe Bend NMP                               0                    
        Little River Canyon NPRES                        0                    
        Natchez Trace PKWY                         583,028           -9.08%   
        Russell Cave NM                                  0                    
...                                                    ...              ...   
        Great Sand Dunes NP & PRES                     470            0.00%   
        Hovenweep NM                                   132            0.00%   
        Mesa Verde NP                                7,519          -25.41%   
        Rocky Mountain NP                            8,750            0.00%   
        Sand Creek Massacre NHS                          0                    

                                   Recreation Visitor Hours Percent Change.3  \
State   Park                                                                   
Alabama                                                                        
        

In [58]:
visitorCount_19_select = visitorCount_19[['Recreation Visitors', 'Percent Change', 'Recreation Visitor Days',
       'Percent Change.1', 'Non-recreation Visitors', 'Percent Change.2',
       'Recreation Visitor Hours', 'Percent Change.3',
       'Non-recreation Visitor Hours', 'Percent Change.4',]]

visitorCount_19_select

Recreation Visitors Percent Change  \
State   Park                                                                   
Alabama                                                                        
        Horseshoe Bend NMP                             28,600        -36.97%   
        Little River Canyon NPRES                     802,375         23.44%   
        Natchez Trace PKWY                            428,736         -2.72%   
        Russell Cave NM                                 8,462        -49.15%   
...                                                       ...            ...   
        Grand Teton NP                              3,289,638         -3.41%   
        John D. Rockefeller, Jr. MEM PKWY           1,373,817         -3.70%   
        Yellowstone NP                              1,865,088         -5.32%   
        Subtotal Wyoming                            7,089,437         -4.60%   
Total   Grand Total                               237,064,318        -27.62%   

                                          Recreation Visitor Days  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                  8,788   
        Little River Canyon NPRES                         168,827   
        Natchez Trace PKWY                                143,413   
        Russell Cave NM                                     1,074   
...                                                           ...   
        Grand Teton NP                                  1,486,410   
        John D. Rockefeller, Jr. MEM PKWY                 144,764   
        Yellowstone NP                                  3,125,466   
        Subtotal Wyoming                                4,883,337   
Total   Grand Total                                    87,912,527   

                                          Percent Change.1  \
State   Park                                                 
Alabama                                                      
        Horseshoe Bend NMP                         -25.20%   
        Little River Canyon NPRES                   27.60%   
        Natchez Trace PKWY                          -3.01%   
        Russell Cave NM                            -55.36%   
...                                                    ...   
        Grand Teton NP                             -15.87%   
        John D. Rockefeller, Jr. MEM PKWY            2.19%   
        Yellowstone NP                             -32.55%   
        Subtotal Wyoming                           -27.08%   
Total   Grand Total                                -26.23%   

                                          Non-recreation Visitors  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                      0   
        Little River Canyon NPRES                               0   
        Natchez Trace PKWY                                583,028   
        Russell Cave NM                                         0   
...                                                           ...   
        Grand Teton NP                                  1,220,029   
        John D. Rockefeller, Jr. MEM PKWY                  10,800   
        Yellowstone NP                                  1,099,227   
        Subtotal Wyoming                                2,334,598   
Total   Grand Total                                   148,436,337   

                                          Percent Change.2  \
State   Park                                                 
Alabama                                                      
        Horseshoe Bend NMP                                   
        Little River Canyon NPRES                            
        Natchez Trace PKWY                          -9.08%   
        Russell Cave NM   

In [97]:
visCount_19 = visitorCount_19_select[['Recreation Visitors', 'Recreation Visitor Days',
       'Non-recreation Visitors',
       'Recreation Visitor Hours',
       'Non-recreation Visitor Hours']]
visCount_19

Recreation Visitors  \
State   Park                                                    
Alabama                                                         
        Horseshoe Bend NMP                             28,600   
        Little River Canyon NPRES                     802,375   
        Natchez Trace PKWY                            428,736   
        Russell Cave NM                                 8,462   
...                                                       ...   
        Grand Teton NP                              3,289,638   
        John D. Rockefeller, Jr. MEM PKWY           1,373,817   
        Yellowstone NP                              1,865,088   
        Subtotal Wyoming                            7,089,437   
Total   Grand Total                               237,064,318   

                                          Recreation Visitor Days  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                  8,788   
        Little River Canyon NPRES                         168,827   
        Natchez Trace PKWY                                143,413   
        Russell Cave NM                                     1,074   
...                                                           ...   
        Grand Teton NP                                  1,486,410   
        John D. Rockefeller, Jr. MEM PKWY                 144,764   
        Yellowstone NP                                  3,125,466   
        Subtotal Wyoming                                4,883,337   
Total   Grand Total                                    87,912,527   

                                          Non-recreation Visitors  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                      0   
        Little River Canyon NPRES                               0   
        Natchez Trace PKWY                                583,028   
        Russell Cave NM                                         0   
...                                                           ...   
        Grand Teton NP                                  1,220,029   
        John D. Rockefeller, Jr. MEM PKWY                  10,800   
        Yellowstone NP                                  1,099,227   
        Subtotal Wyoming                                2,334,598   
Total   Grand Total                                   148,436,337   

                                          Recreation Visitor Hours  \
State   Park                                                         
Alabama                                                              
        Horseshoe Bend NMP                                 105,461   
        Little River Canyon NPRES                        2,025,934   
        Natchez Trace PKWY                               1,720,966   
        Russell Cave NM                                     12,894   
...                                                            ...   
        Grand Teton NP                                  17,836,930   
        John D. Rockefeller, Jr. MEM PKWY                1,737,179   
        Yellowstone NP                                  37,505,599   
        Subtotal Wyoming                                58,600,107   
Total   Grand Total                                  1,054,952,561   

                                          Non-recreation Visitor Hours  
State   Park                                                            
Alabama                                                                 
        Horseshoe Bend NMP                                           0  
        Little River Canyon NPRES                                    0  
        Natchez Trace PKWY                                     583,028  
        Russell Cave NM                            

In [59]:
path = os.path.join("data/visitors_2018.csv")
visitorCount_18 = pd.read_csv(path, header=[7], encoding='utf-8', delimiter=",", index_col=[0,1], keep_default_na=False, na_filter=True)
visitorCount_18 = visitorCount_20.dropna(axis=1, how='all', thresh=3, inplace=False)

visitorCount_18.head(100)

Recreation Visitors Percent Change  \
State   Park                                                            
Alabama                                                                 
        Horseshoe Bend NMP                      28,600        -36.97%   
        Little River Canyon NPRES              802,375         23.44%   
        Natchez Trace PKWY                     428,736         -2.72%   
        Russell Cave NM                          8,462        -49.15%   
...                                                ...            ...   
        Great Sand Dunes NP & PRES             461,532        -12.51%   
        Hovenweep NM                             8,736        -43.92%   
        Mesa Verde NP                          287,477        -48.31%   
        Rocky Mountain NP                    3,305,199        -29.23%   
        Sand Creek Massacre NHS                  4,215        -26.07%   

                                   Recreation Visitor Days Percent Change.1  \
State   Park                                                                  
Alabama                                                                       
        Horseshoe Bend NMP                           8,788          -25.20%   
        Little River Canyon NPRES                  168,827           27.60%   
        Natchez Trace PKWY                         143,413           -3.01%   
        Russell Cave NM                              1,074          -55.36%   
...                                                    ...              ...   
        Great Sand Dunes NP & PRES                 159,822          -29.39%   
        Hovenweep NM                                 2,350          -48.12%   
        Mesa Verde NP                              156,122          -52.80%   
        Rocky Mountain NP                        1,799,117          -31.54%   
        Sand Creek Massacre NHS                        702          -26.11%   

                                   Unnamed: 6 Unnamed: 7  \
State   Park                                               
Alabama                                                    
        Horseshoe Bend NMP                                 
        Little River Canyon NPRES                          
        Natchez Trace PKWY                                 
        Russell Cave NM                                    
...                                       ...        ...   
        Great Sand Dunes NP & PRES                         
        Hovenweep NM                                       
        Mesa Verde NP                                      
        Rocky Mountain NP                                  
        Sand Creek Massacre NHS                            

                                   Non-recreation Visitors Percent Change.2  \
State   Park                                                                  
Alabama                                                                       
        Horseshoe Bend NMP                               0                    
        Little River Canyon NPRES                        0                    
        Natchez Trace PKWY                         583,028           -9.08%   
        Russell Cave NM                                  0                    
...                                                    ...              ...   
        Great Sand Dunes NP & PRES                     470            0.00%   
        Hovenweep NM                                   132            0.00%   
        Mesa Verde NP                                7,519          -25.41%   
        Rocky Mountain NP                            8,750            0.00%   
        Sand Creek Massacre NHS                          0                    

                                   Recreation Visitor Hours Percent Change.3  \
State   Park                                                                   
Alabama                                                                        
        

In [61]:
visitorCount_18_select = visitorCount_18[['Recreation Visitors', 'Percent Change', 'Recreation Visitor Days',
       'Percent Change.1', 'Non-recreation Visitors', 'Percent Change.2',
       'Recreation Visitor Hours', 'Percent Change.3',
       'Non-recreation Visitor Hours', 'Percent Change.4',]]

visitorCount_18_select

Recreation Visitors Percent Change  \
State   Park                                                                   
Alabama                                                                        
        Horseshoe Bend NMP                             28,600        -36.97%   
        Little River Canyon NPRES                     802,375         23.44%   
        Natchez Trace PKWY                            428,736         -2.72%   
        Russell Cave NM                                 8,462        -49.15%   
...                                                       ...            ...   
        Grand Teton NP                              3,289,638         -3.41%   
        John D. Rockefeller, Jr. MEM PKWY           1,373,817         -3.70%   
        Yellowstone NP                              1,865,088         -5.32%   
        Subtotal Wyoming                            7,089,437         -4.60%   
Total   Grand Total                               237,064,318        -27.62%   

                                          Recreation Visitor Days  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                  8,788   
        Little River Canyon NPRES                         168,827   
        Natchez Trace PKWY                                143,413   
        Russell Cave NM                                     1,074   
...                                                           ...   
        Grand Teton NP                                  1,486,410   
        John D. Rockefeller, Jr. MEM PKWY                 144,764   
        Yellowstone NP                                  3,125,466   
        Subtotal Wyoming                                4,883,337   
Total   Grand Total                                    87,912,527   

                                          Percent Change.1  \
State   Park                                                 
Alabama                                                      
        Horseshoe Bend NMP                         -25.20%   
        Little River Canyon NPRES                   27.60%   
        Natchez Trace PKWY                          -3.01%   
        Russell Cave NM                            -55.36%   
...                                                    ...   
        Grand Teton NP                             -15.87%   
        John D. Rockefeller, Jr. MEM PKWY            2.19%   
        Yellowstone NP                             -32.55%   
        Subtotal Wyoming                           -27.08%   
Total   Grand Total                                -26.23%   

                                          Non-recreation Visitors  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                      0   
        Little River Canyon NPRES                               0   
        Natchez Trace PKWY                                583,028   
        Russell Cave NM                                         0   
...                                                           ...   
        Grand Teton NP                                  1,220,029   
        John D. Rockefeller, Jr. MEM PKWY                  10,800   
        Yellowstone NP                                  1,099,227   
        Subtotal Wyoming                                2,334,598   
Total   Grand Total                                   148,436,337   

                                          Percent Change.2  \
State   Park                                                 
Alabama                                                      
        Horseshoe Bend NMP                                   
        Little River Canyon NPRES                            
        Natchez Trace PKWY                          -9.08%   
        Russell Cave NM   

In [94]:
visCount_18 = visitorCount_18_select[['Recreation Visitors', 'Recreation Visitor Days',
       'Non-recreation Visitors',
       'Recreation Visitor Hours',
       'Non-recreation Visitor Hours']]
visCount_18

Recreation Visitors  \
State   Park                                                    
Alabama                                                         
        Horseshoe Bend NMP                             28,600   
        Little River Canyon NPRES                     802,375   
        Natchez Trace PKWY                            428,736   
        Russell Cave NM                                 8,462   
...                                                       ...   
        Grand Teton NP                              3,289,638   
        John D. Rockefeller, Jr. MEM PKWY           1,373,817   
        Yellowstone NP                              1,865,088   
        Subtotal Wyoming                            7,089,437   
Total   Grand Total                               237,064,318   

                                          Recreation Visitor Days  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                  8,788   
        Little River Canyon NPRES                         168,827   
        Natchez Trace PKWY                                143,413   
        Russell Cave NM                                     1,074   
...                                                           ...   
        Grand Teton NP                                  1,486,410   
        John D. Rockefeller, Jr. MEM PKWY                 144,764   
        Yellowstone NP                                  3,125,466   
        Subtotal Wyoming                                4,883,337   
Total   Grand Total                                    87,912,527   

                                          Non-recreation Visitors  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                      0   
        Little River Canyon NPRES                               0   
        Natchez Trace PKWY                                583,028   
        Russell Cave NM                                         0   
...                                                           ...   
        Grand Teton NP                                  1,220,029   
        John D. Rockefeller, Jr. MEM PKWY                  10,800   
        Yellowstone NP                                  1,099,227   
        Subtotal Wyoming                                2,334,598   
Total   Grand Total                                   148,436,337   

                                          Recreation Visitor Hours  \
State   Park                                                         
Alabama                                                              
        Horseshoe Bend NMP                                 105,461   
        Little River Canyon NPRES                        2,025,934   
        Natchez Trace PKWY                               1,720,966   
        Russell Cave NM                                     12,894   
...                                                            ...   
        Grand Teton NP                                  17,836,930   
        John D. Rockefeller, Jr. MEM PKWY                1,737,179   
        Yellowstone NP                                  37,505,599   
        Subtotal Wyoming                                58,600,107   
Total   Grand Total                                  1,054,952,561   

                                          Non-recreation Visitor Hours  
State   Park                                                            
Alabama                                                                 
        Horseshoe Bend NMP                                           0  
        Little River Canyon NPRES                                    0  
        Natchez Trace PKWY                                     583,028  
        Russell Cave NM                            

In [89]:
path = os.path.join("data/visitors_2017.csv")
visitorCount_17 = pd.read_csv(path, header=[6], encoding='utf-8', delimiter=",", index_col=[0,1], keep_default_na=False, na_filter=True)
visitorCount_17 = visitorCount_17.dropna(axis=0, how='all', thresh=3, inplace=False)

visitorCount_17.head(100)

Recreation Visitors  \
State   Park                                             
Alabama                                                  
        Horseshoe Bend NMP                      74,255   
        Little River Canyon NPRES              367,058   
        Natchez Trace PKWY                     442,825   
        Russell Cave NM                         24,377   
...                                                ...   
        Great Sand Dunes NP & PRES             486,935   
        Hovenweep NM                            17,588   
        Mesa Verde NP                          613,788   
        Rocky Mountain NP                    4,437,215   
        Sand Creek Massacre NHS                  6,535   

                                   Recreation Visitor Days  \
State   Park                                                 
Alabama                                                      
        Horseshoe Bend NMP                          15,009   
        Little River Canyon NPRES                   80,380   
        Natchez Trace PKWY                         148,659   
        Russell Cave NM                              3,243   
...                                                    ...   
        Great Sand Dunes NP & PRES                 192,361   
        Hovenweep NM                                 5,365   
        Mesa Verde NP                              333,353   
        Rocky Mountain NP                        2,538,724   
        Sand Creek Massacre NHS                      1,089   

                                   Non-recreation Visitors  \
State   Park                                                 
Alabama                                                      
        Horseshoe Bend NMP                       1,014,144   
        Little River Canyon NPRES                        0   
        Natchez Trace PKWY                         613,012   
        Russell Cave NM                                  0   
...                                                    ...   
        Great Sand Dunes NP & PRES                     470   
        Hovenweep NM                                   132   
        Mesa Verde NP                                9,012   
        Rocky Mountain NP                            8,750   
        Sand Creek Massacre NHS                          0   

                                   Recreation Visitor Hours Unnamed: 6  \
State   Park                                                             
Alabama                                                                  
        Horseshoe Bend NMP                          180,109              
        Little River Canyon NPRES                   964,562              
        Natchez Trace PKWY                        1,783,918              
        Russell Cave NM                              38,927              
...                                                     ...        ...   
        Great Sand Dunes NP & PRES                2,308,335              
        Hovenweep NM                                 64,384              
        Mesa Verde NP                             4,000,241              
        Rocky Mountain NP                        30,464,697              
        Sand Creek Massacre NHS                      13,070              

                                   Unnamed: 7 Non-recreation Visitor Hours  \
State   Park                                                                 
Alabama                                                                      
        Horseshoe Bend NMP                                         338,014   
        Little River Canyon NPRES                                        0   
        Natchez Trace PKWY                                         613,012   
        Russell Cave NM                                                  0   
...                                       ...                          ...   
        Great Sand Dunes NP & PRES                                     470   
 

In [90]:
visitorCount_17_select = visitorCount_17[['Recreation Visitors', 'Recreation Visitor Days',
       'Non-recreation Visitors',
       'Recreation Visitor Hours',
       'Non-recreation Visitor Hours']]

visitorCount_17_select

Recreation Visitors  \
State   Park                                                    
Alabama                                                         
        Horseshoe Bend NMP                             74,255   
        Little River Canyon NPRES                     367,058   
        Natchez Trace PKWY                            442,825   
        Russell Cave NM                                24,377   
...                                                       ...   
        Grand Teton NP                              3,317,000   
        John D. Rockefeller, Jr. MEM PKWY           1,433,292   
        Yellowstone NP                              2,017,098   
        Subtotal Wyoming                            7,456,556   
Total   Grand Total                               330,882,772   

                                          Recreation Visitor Days  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                                 15,009   
        Little River Canyon NPRES                          80,380   
        Natchez Trace PKWY                                148,659   
        Russell Cave NM                                     3,243   
...                                                           ...   
        Grand Teton NP                                  1,871,697   
        John D. Rockefeller, Jr. MEM PKWY                 146,216   
        Yellowstone NP                                  4,900,039   
        Subtotal Wyoming                                7,087,235   
Total   Grand Total                                   120,580,704   

                                          Non-recreation Visitors  \
State   Park                                                        
Alabama                                                             
        Horseshoe Bend NMP                              1,014,144   
        Little River Canyon NPRES                               0   
        Natchez Trace PKWY                                613,012   
        Russell Cave NM                                         0   
...                                                           ...   
        Grand Teton NP                                  1,652,347   
        John D. Rockefeller, Jr. MEM PKWY                  10,800   
        Yellowstone NP                                  1,218,042   
        Subtotal Wyoming                                2,888,592   
Total   Grand Total                                   172,112,734   

                                          Recreation Visitor Hours  \
State   Park                                                         
Alabama                                                              
        Horseshoe Bend NMP                                 180,109   
        Little River Canyon NPRES                          964,562   
        Natchez Trace PKWY                               1,783,918   
        Russell Cave NM                                     38,927   
...                                                            ...   
        Grand Teton NP                                  22,460,367   
        John D. Rockefeller, Jr. MEM PKWY                1,754,592   
        Yellowstone NP                                  58,800,471   
        Subtotal Wyoming                                85,046,838   
Total   Grand Total                                  1,446,970,692   

                                          Non-recreation Visitor Hours  
State   Park                                                            
Alabama                                                                 
        Horseshoe Bend NMP                                     338,014  
        Little River Canyon NPRES                                    0  
        Natchez Trace PKWY                                     613,012  
        Russell Cave NM                            

In [119]:
visCount_comb_1 = visCount_18.join(visitorCount_17_select, on=['State', 'Park'], lsuffix='_17', rsuffix='_18')
visCount_comb.head(50)

Natchez Trace PKWY                                  1,138,450   
        Natchez Trace PKWY                                  1,138,450   
        Natchez Trace PKWY                                    613,012   
        Natchez Trace PKWY                                    613,012   
        Natchez Trace PKWY                                    613,012   
        Natchez Trace PKWY                                  7,005,845   
        Natchez Trace PKWY                                  7,005,845   
        Natchez Trace PKWY                                  7,005,845   
        Natchez Trace PKWY                                  1,138,450   
        Natchez Trace PKWY                                  1,138,450   
        Natchez Trace PKWY                                  1,138,450   
        Russell Cave NM                                             0   
        Tuskegee Airmen NHS                                         0   
        Tuskegee Institute NHS                                      0   
        Subtotal Alabama                                      951,026   
Alaska                                                                  
        Aniakchak NM & PRES                                         0   
        Bering Land Bridge NPRES                                9,024   
        Cape Krusenstern NM                                    60,000   
        Denali NP & PRES                                      198,321   
        Gates of the Arctic NP & PRES                               0   
        Glacier Bay NP & PRES                                  97,380   
        Katmai NP & PRES                                            0   
        Kenai Fjords NP                                             0   
        Klondike Gold Rush NHP Alaska                               0   
        Kobuk Valley NP                                        62,000   
        Lake Clark NP & PRES                                        0   
        Noatak NPRES                                           68,000   
        Sitka NHP                                                   0   
        Wrangell-St. Elias NP & PRES                                0   
        Yukon-Charley Rivers NPRES                                  0   

                                      Recreation Visitors_18  \
State   Park                                                   
Alabama                                                        
        Horseshoe Bend NMP                            28,600   
        Little River Canyon NPRES                    802,375   
        Natchez Trace PKWY                           428,736   
        Natchez Trace PKWY                         4,899,845   
        Natchez Trace PKWY                           796,225   
        Natchez Trace PKWY                           428,736   
        Natchez Trace PKWY                         4,899,845   
        Natchez Trace PKWY                           796,225   
        Natchez Trace PKWY                           428,736   
        Natchez Trace PKWY                         4,899,845   
        Natchez Trace PKWY                           796,225   
        Natchez Trace PKWY                           428,736   
        Natchez Trace PKWY                         4,899,845   
        Natchez Trace PKWY                           796,225   
        Natchez Trace PKWY                           428,736   
        Natchez Trace PKWY                         4,899,845   
        Natchez Trace PKWY                           796,225   
        Natchez Trace PKWY                           428,736   
        Natchez Trace PKWY                         4,899,845   
        Natchez Trace PKWY                           796,225   
        Natchez Trace PKWY                           428,736   
        Natchez Trace PKWY                         4,899,845   
        Natchez Trace PKWY                           796,225   
        Natchez Trace PKWY                           428,736   
        Natchez 

In [120]:
visCount_comb_2 = visCount_20.join(visCount_19, on=['State', 'Park'], lsuffix='_20', rsuffix='_19')
visCount_comb_2.head(50)

ate          Park                                                     
Alabama                                                                 
               Horseshoe Bend NMP                              28,600   
               Little River Canyon NPRES                      802,375   
               Natchez Trace PKWY                             428,736   
               Natchez Trace PKWY                           4,899,845   
               Natchez Trace PKWY                             796,225   
               Russell Cave NM                                  8,462   
               Tuskegee Airmen NHS                              6,843   
               Tuskegee Institute NHS                           4,737   
               Subtotal Alabama                             1,279,753   
Alaska                                                                  
               Aniakchak NM & PRES                                 36   
               Bering Land Bridge NPRES                         2,642   
               Cape Krusenstern NM                             11,369   
               Denali NP & PRES                                54,850   
               Gates of the Arctic NP & PRES                    2,872   
               Glacier Bay NP & PRES                            5,748   
               Katmai NP & PRES                                51,511   
               Kenai Fjords NP                                115,882   
               Klondike Gold Rush NHP Alaska                      297   
               Kobuk Valley NP                                 11,185   
               Lake Clark NP & PRES                             4,948   
               Noatak NPRES                                    12,533   
               Sitka NHP                                      115,494   
               Wrangell-St. Elias NP & PRES                    16,655   
               Yukon-Charley Rivers NPRES                         666   
               Subtotal Alaska                                406,688   
American Samoa                                                          
               National Park of American Samoa                  4,819   
               Subtotal American Samoa                          4,819   
Arizona                                                                 
               Canyon de Chelly NM                             76,752   
               Casa Grande Ruins NM                            44,269   
               Chiricahua NM                                   44,794   
               Coronado NMEM                                  156,199   
               Fort Bowie NHS                                   6,456   
               Glen Canyon NRA                                357,804   
               Glen Canyon NRA                              2,195,589   
               Grand Canyon NP                              2,897,098   
               Hubbell Trading Post NHS                        11,407   
               Lake Mead NRA                                2,004,127   
               Lake Mead NRA                                6,012,382   
               Montezuma Castle NM                            242,028   
               Navajo NM                                        3,985   
               Organ Pipe Cactus NM                           191,512   
               Petrified Forest NP                            384,483   
               Pipe Spring NM                                  10,623   
               Saguaro NP                                     762,226   
               Sunset Crater Volcano NM                        83,669   
               Tonto NM                                        24,756   

                                               Recreation Visitor Days_19  \
State          Park                                                         
Alabama                                                                     
               Horseshoe Bend NMP              

In [121]:
visCount_comb = visCount_comb_2.join(visCount_comb_1, on=['State', 'Park'])
visCount_comb.head(50)

Natchez Trace PKWY                                  1,082,765   
        Natchez Trace PKWY                                  1,082,765   
        Natchez Trace PKWY                                    583,028   
        Natchez Trace PKWY                                    583,028   
        Natchez Trace PKWY                                    583,028   
        Natchez Trace PKWY                                  6,663,162   
        Natchez Trace PKWY                                  6,663,162   
        Natchez Trace PKWY                                  6,663,162   
        Natchez Trace PKWY                                  1,082,765   
        Natchez Trace PKWY                                  1,082,765   
        Natchez Trace PKWY                                  1,082,765   
        Russell Cave NM                                             0   
        Tuskegee Airmen NHS                                         0   
        Tuskegee Institute NHS                                      0   
        Subtotal Alabama                                      583,028   
Alaska                                                                  
        Aniakchak NM & PRES                                         0   
        Bering Land Bridge NPRES                               20,592   
        Cape Krusenstern NM                                   171,840   
        Denali NP & PRES                                      139,003   
        Gates of the Arctic NP & PRES                               0   
        Glacier Bay NP & PRES                                  48,200   
        Katmai NP & PRES                                       13,280   
        Kenai Fjords NP                                             0   
        Klondike Gold Rush NHP Alaska                               0   
        Kobuk Valley NP                                       199,528   
        Lake Clark NP & PRES                                        0   
        Noatak NPRES                                          162,640   
        Sitka NHP                                                   0   
        Wrangell-St. Elias NP & PRES                                0   
        Yukon-Charley Rivers NPRES                              7,800   

                                      Recreation Visitors_18  \
State   Park                                                   
Alabama                                                        
        Horseshoe Bend NMP                            74,255   
        Little River Canyon NPRES                    367,058   
        Natchez Trace PKWY                           442,825   
        Natchez Trace PKWY                         5,060,852   
        Natchez Trace PKWY                           822,391   
        Natchez Trace PKWY                           442,825   
        Natchez Trace PKWY                         5,060,852   
        Natchez Trace PKWY                           822,391   
        Natchez Trace PKWY                           442,825   
        Natchez Trace PKWY                         5,060,852   
        Natchez Trace PKWY                           822,391   
        Natchez Trace PKWY                           442,825   
        Natchez Trace PKWY                         5,060,852   
        Natchez Trace PKWY                           822,391   
        Natchez Trace PKWY                           442,825   
        Natchez Trace PKWY                         5,060,852   
        Natchez Trace PKWY                           822,391   
        Natchez Trace PKWY                           442,825   
        Natchez Trace PKWY                         5,060,852   
        Natchez Trace PKWY                           822,391   
        Natchez Trace PKWY                           442,825   
        Natchez Trace PKWY                         5,060,852   
        Natchez Trace PKWY                           822,391   
        Natchez Trace PKWY                           442,825   
        Natchez 

In [122]:
path = os.path.join("data/NP_trafficcounts_2020.csv")
NP_traffic = pd.read_csv(path, header=[3], encoding='utf-8', delimiter=",", index_col=[0,1], keep_default_na=False, na_filter=True)
NP_traffic  = visitorCount_17.dropna(axis=0, how='all', thresh=3, inplace=False)

NP_traffic.head(100)

ParkType          Region  \
ParkName    UnitCode                                             
Acadia NP   ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
...                                        ...             ...   
Amistad NRA AMIS      National Recreation Area  Intermountain    
            AMIS      National Recreation Area  Intermountain    
            AMIS      National Recreation Area  Intermountain    
            AMIS      National Recreation Area  Intermountain    
            AMIS      National Recreation Area  Intermountain    

                                      TrafficCounter  Year  Month  \
ParkName    UnitCode                                                
Acadia NP   ACAD         TRAFFIC COUNT AT SAND BEACH  2020      1   
            ACAD         TRAFFIC COUNT AT SAND BEACH  2020      2   
            ACAD         TRAFFIC COUNT AT SAND BEACH  2020      3   
            ACAD         TRAFFIC COUNT AT SAND BEACH  2020      4   
            ACAD         TRAFFIC COUNT AT SAND BEACH  2020      5   
...                                              ...   ...    ...   
Amistad NRA AMIS      TRAFFIC COUNT AT OLD 277 NORTH  2020     12   
            AMIS        TRAFFIC COUNT AT PECOS RIVER  2020      1   
            AMIS        TRAFFIC COUNT AT PECOS RIVER  2020      2   
            AMIS        TRAFFIC COUNT AT PECOS RIVER  2020      3   
            AMIS        TRAFFIC COUNT AT PECOS RIVER  2020      4   

                     TrafficCount ParkNameTotal UnitCodeTotal  \
ParkName    UnitCode                                            
Acadia NP   ACAD            2,761     Acadia NP          ACAD   
            ACAD            3,072     Acadia NP          ACAD   
            ACAD            5,577     Acadia NP          ACAD   
            ACAD                0     Acadia NP          ACAD   
            ACAD                0     Acadia NP          ACAD   
...                           ...           ...           ...   
Amistad NRA AMIS            2,053   Amistad NRA          AMIS   
            AMIS              571   Amistad NRA          AMIS   
            AMIS              571   Amistad NRA          AMIS   
            AMIS              631   Amistad NRA          AMIS   
            AMIS              631   Amistad NRA          AMIS   

                                 ParkTypeTotal      RegionType  \
ParkName    UnitCode                                             
Acadia NP   ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
...                                        ...             ...   
Amistad NRA AMIS      National Recreation Area  Intermountain    
            AMIS      National Recreation Area  Intermountain    
            AMIS      National Recreation Area  Intermountain    
            AMIS      National Recreation Area  Intermountain    
            AMIS      National Recreation Area  Intermountain    

                                   TrafficCountTotal  YearTotal  \
ParkName    UnitCode                                              
Acadia NP   ACAD         TRAFFIC COUNT AT SAND BEACH       2020   
            ACAD         TRAFFIC COUNT AT SAND BEACH       2020   
            ACAD         TRAFFIC COUNT AT SAND BEACH       2020   
            ACAD         TRAFFIC COUNT AT SAND BEACH       2020   
            ACAD         TRAFFIC COUNT AT SAND BEACH       2020   
...                                              ...        ...   
Amistad NRA AMIS      TRAFFIC COUNT AT O

In [104]:
NP_traffic.columns

Index(['ParkType', 'Region', 'TrafficCounter', 'Year', 'Month', 'TrafficCount',
       'ParkNameTotal', 'UnitCodeTotal', 'ParkTypeTotal', 'RegionType',
       'TrafficCountTotal', 'YearTotal', 'TrafficCountTotal.1'],
      dtype='object')

In [108]:
NP_sorted = NP_traffic[['ParkType', 'Region', 'TrafficCounter', 'Year', 'Month', 'TrafficCount',
       'TrafficCountTotal.1']].rename(columns={'TrafficCountTotal.1': 'TrafficCountTotal'})

NP_sorted.head(25)

ParkType          Region  \
ParkName    UnitCode                                             
Acadia NP   ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
            ACAD                 National Park      Northeast    
Amistad NRA AMIS      National Recreation Area  Intermountain    

                                          TrafficCounter  Year  Month  \
ParkName    UnitCode                                                    
Acadia NP   ACAD             TRAFFIC COUNT AT SAND BEACH  2020      1   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020      2   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020      3   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020      4   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020      5   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020      6   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020      7   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020      8   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020      9   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020     10   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020     11   
            ACAD             TRAFFIC COUNT AT SAND BEACH  2020     12   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020      1   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020      2   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020      3   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020      4   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020      5   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020      6   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020      7   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020      8   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020      9   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020     10   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020     11   
            ACAD               TRAFFIC COUNT AT SCHOODIC  2020     12   
Amistad NRA AMIS      TRAFFIC COUNT AT BLACK BRUSH POINT  2020      1   

                     TrafficCount TrafficCountTotal  
ParkName    UnitCode                                 
Acadia NP   ACAD            2,761           300,632  
            ACAD            3,072           300,632  
            ACAD            5,577           300,632  
            A

In [161]:
NP_parktype = NP_sorted[["ParkType"]] == 'National Park'
NP_parktype

ParkType
ParkName  UnitCode          
Acadia NP ACAD          True
          ACAD          True
          ACAD          True
          ACAD          True
          ACAD          True
...                      ...
Zion NP   ZION          True
          ZION          True
          ZION          True
          ZION          True
          ZION          True

[7623 rows x 1 columns]

In [164]:
path = os.path.join("data/NP_publicuse_20y.csv")
NP_pubUse = pd.read_csv(path, header=[3], encoding='utf-8', delimiter=",", index_col=[0,1], keep_default_na=False, na_filter=True)
NP_pubUse  = NP_pubUse.dropna(axis=0, how='all', thresh=3, inplace=False)

NP_pubUse.head(100)

ParkType      Region State  Year  Month  \
ParkName  UnitCode                                                 
Acadia NP ACAD      National Park  Northeast     ME  2000      1   
          ACAD      National Park  Northeast     ME  2000      2   
          ACAD      National Park  Northeast     ME  2000      3   
          ACAD      National Park  Northeast     ME  2000      4   
          ACAD      National Park  Northeast     ME  2000      5   
...                           ...         ...   ...   ...    ...   
          ACAD      National Park  Northeast     ME  2007     12   
          ACAD      National Park  Northeast     ME  2008      1   
          ACAD      National Park  Northeast     ME  2008      2   
          ACAD      National Park  Northeast     ME  2008      3   
          ACAD      National Park  Northeast     ME  2008      4   

                   RecreationVisits NonRecreationVisits RecreationHours  \
ParkName  UnitCode                                                        
Acadia NP ACAD               22,500              20,373         148,500   
          ACAD               23,848              19,923         157,397   
          ACAD               35,423              20,373         233,792   
          ACAD               78,555              21,723         455,619   
          ACAD              166,456              21,923         965,445   
...                             ...                 ...             ...   
          ACAD                9,070                 600          54,418   
          ACAD               10,744                 600          70,910   
          ACAD               11,131                 600          73,463   
          ACAD               17,489                 600         115,426   
          ACAD               53,798                 600         312,027   

                   NonRecreationHours ConcessionerLodging  ...  \
ParkName  UnitCode                                         ...   
Acadia NP ACAD                 20,373                   0  ...   
          ACAD                 19,923                   0  ...   
          ACAD                 20,373                   0  ...   
          ACAD                 21,723                   0  ...   
          ACAD                 21,923                   0  ...   
...                               ...                 ...  ...   
          ACAD                    600                   0  ...   
          ACAD                    600                   0  ...   
          ACAD                    600                   0  ...   
          ACAD                    600                   0  ...   
          ACAD                    600                   0  ...   

                   NonRecreationVisitsTotal RecreationHoursTotal  \
ParkName  UnitCode                                                 
Acadia NP ACAD                      252,576           13,230,679   
          ACAD                      252,576           13,230,679   
          ACAD                      252,576           13,230,679   
          ACAD                      252,576           13,230,679   
          ACAD                      252,576           13,230,679   
...                                     ...                  ...   
          ACAD                       47,100           11,331,597   
          ACAD                       47,100           10,635,395   
          ACAD                       47,100           10,635,395   
          ACAD                       47,100           10,635,395   
          ACAD                       47,100           10,635,395   

                   NonRecreationHoursTotal ConcessionerLodgingTotal  \
ParkName  UnitCode                                                    
Acadia NP ACAD                     252,576                        0   
          ACAD                     252,576                        0   
          ACAD                     252,576                        0   
          ACAD                     252,576                       